## Parallel workloads archive parser

Authors of this archive wrote article where explained all problems they faced -> http://www.cs.huji.ac.il/~feit/papers/PWA14JPDC.pdf

Authors book about Workload Modeling -> http://www.cs.huji.ac.il/~feit/wlmod/wlmod.pdf

Parallel workloads archive link -> http://www.cs.huji.ac.il/labs/parallel/workload/index.html

Logs Format -> http://www.cs.huji.ac.il/labs/parallel/workload/swf.html

Logs -> http://www.cs.huji.ac.il/labs/parallel/workload/logs.html

Log Parser on Perl -> http://www.cs.huji.ac.il/labs/parallel/workload/parse_swf.pl

## Standard workload format
- **Job Number** -- a counter field, starting from 1.

- **Submit Time** -- in seconds. The earliest time the log refers to is zero, and is usually the submittal time of the first job. The lines in the log are sorted by ascending submittal times. It makes sense for jobs to also be numbered in this order.

- **Wait Time** -- in seconds. The difference between the job's submit time and the time at which it actually began to run. Naturally, this is only relevant to real logs, not to models.

- **Run Time** -- in seconds. The wall clock time the job was running (end time minus start time). We decided to use "wait time" and "run time" instead of the equivalent "start time" and "end time" because they are directly attributable to the scheduler and application, and are more suitable for models where only the run time is relevant. <font color='red'> Note that when values are rounded to an integral number of seconds (as often happens in logs) a run time of 0 is possible and means the job ran for less than 0.5 seconds. On the other hand it is permissable to use floating point values for time fields. </font>

- **Number of Allocated Processors** -- an integer. In most cases this is also the number of processors the job uses; if the job does not use all of them, we typically don't know about it.

- **Average CPU Time Used** -- both user and system, in seconds. This is the average over all processors of the CPU time used, and may therefore be smaller than the wall clock runtime. If a log contains the total CPU time used by all the processors, it is divided by the number of allocated processors to derive the average.

- **Used Memory** -- in kilobytes. This is again the average per processor.

- **Requested Number of Processors**.

- **Requested Time** -- This can be either runtime (measured in wallclock seconds), or average CPU time per processor (also in seconds) -- the exact meaning is determined by a header comment. In many logs this field is used for the user runtime estimate (or upper bound) used in backfilling. If a log contains a request for total CPU time, it is divided by the number of requested processors.

- **Requested Memory (again kilobytes per processor).**

- **Status** 1 if the job was completed, 0 if it failed, and 5 if cancelled. If information about chekcpointing or swapping is included, other values are also possible. See usage note -> (http://www.cs.huji.ac.il/labs/parallel/workload/swf.html). This field is meaningless for models, so would be -1.
    - The main usage of the status field is to note the job's status. This isn't as straightforward as it sounds. 
    - The simple case is jobs that complete normally, and have status 1.
    - The harder case is jobs that don't complete normally. This can happen for several reasons:
        - The job failed (e.g. segmentation fault). This is given status 0.
        - The job was cancelled by the user (like ^C in Unix). This is given status 5. Note that cancelled jobs may have positive runtimes and processors if cancelled after they started to run, or 0 or -1 if cancelled while waiting in the queue.
        - The job was killed by the system (e.g. because it exceeded its requested run time). This may be given different status values in different logs; it will typically be 0 or 5, but might also be 1.
        
        Note also that the distinction between failure / cancellation / killing is not necessarily accurate, as the distinction typically does not appear in the original logs. If a log contains information about checkpoints and swapping out of jobs, a job can have multiple lines in the log. In fact, we propose that the job information appear twice. First, there will be one line that summarizes the whole job: its submit time is the submit time of the job, its runtime is the sum of all partial runtimes, and its code is 0 or 1 according to the completion status of the whole job. In addition, there will be separate lines for each instance of partial execution between being swapped out. All these lines have the same job ID and appear consecutively in the log. Only the first has a submit time; the rest only have a wait time since the previous burst. The completed code for all these lines is 2, meaning "to be continued"; the completion code for the last such line is 3 or 4, corresponding to completion or being killed. It should be noted that such details are only useful for studying the behavior of the logged system, and are not a feature of the workload. Such studies should ignore lines with completion codes of 0 and 1, and only use lines with 2, 3, and 4. <font color="red">For workload studies, only the single-line summary of the job should be used, as identified by a code of 0 or 1. </font>

- **User ID** -- a natural number, between one and the number of different users.

- **Group ID** -- a natural number, between one and the number of different groups. Some systems control resource usage by groups rather than by individual users.

- **Executable (Application) Number** -- a natural number, between one and the number of different applications appearing in the workload. In some logs, this might represent a script file used to run jobs rather than the executable directly; this should be noted in a header comment.

- **Queue Number** -- a natural number, between one and the number of different queues in the system. The nature of the system's queues should be explained in a header comment. This field is where batch and interactive jobs should be differentiated: we suggest the convention of denoting interactive jobs by 0.

- **Partition Number** -- a natural number, between one and the number of different partitions in the systems. The nature of the system's partitions should be explained in a header comment. For example, it is possible to use partition numbers to identify which machine in a cluster was used.

- **Preceding Job Number** -- this is the number of a previous job in the workload, such that the current job can only start after the termination of this preceding job. Together with the next field, this allows the workload to include feedback as described below.

- **Think Time from Preceding Job** -- this is the number of seconds that should elapse between the termination of the preceding job and the submittal of this one.


## Header Comments
- **Version**: Version number of the standard format the file uses. The format described here is version 2.
- **Computer**: Brand and model of computer
- **Installation**: Location of installation and machine name
- **Acknowledge**: Name of person(s) to acknowledge for creating/collecting the workload.
- **Information**: Web site or email that contain more information about the workload or installation.
- **Conversion**: Name and email of whoever converted the log to the standard format.
- **MaxJobs**: Integer, total number of jobs in this workload file.
- **MaxRecords**: Integer, total number of records in this workload file. If no checkpointing/swapping information is included, there is one record per job, and this is equal to MaxJobs. But with chekpointing/swapping there may be multiple records per job.
- **Preemption**: Enumerated, with four possible values. 'No' means that jobs run to completion, and are represented by a single line in the file. 'Yes' means that the execution of a job may be split into several parts, and each is represented by a separate line. 'Double' means that jobs may be split, and their information appears twice in the file: once as a one-line summary, and again as a sequence of lines representing the parts, as suggested above. 'TS' means time slicing is used, but no details are available.
- **UnixStartTime**: When the log starts, in Unix time (seconds since the epoch)
- **TimeZone**: DEPRECATED and replaced by TimeZoneString.
- A value to add to times given as seconds since the epoch. The sum can then be fed into gmtime (Greenwich time function) to get the correct date and hour of the day. The default is 0, and then gmtime can be used directly. - Note: do not use localtime, as then the results will depend on the difference between your time zone and the installation time zone.
- **TimeZoneString**: Replaces the buggy and now deprecated TimeZone. TimeZoneString is a standard UNIX string indicating the time zone in which the log was generated; this is actually the name of a zoneinfo file, e.g. "Europe/Paris". All times within the SWF file are in this time zone. For more details see the usage note below.
- **StartTime**: When the log starts, in human readable form, in this standard format: Tue Feb 21 18:44:15 IST 2006 (as printed by the UNIX 'date' utility).
- **EndTime**: When the log ends (the last termination), formatted like StartTime.
- **MaxNodes**: Integer, number of nodes in the computer. List the number of nodes in different partitions in parentheses if applicable.
- **MaxProcs**: Integer, number of processors in the computer. This is different from MaxNodes if each node is an SMP. List the number of processors in different partitions in parentheses if applicable.
- **MaxRuntime**: Integer, in seconds. This is the maximum that the system allowed, and may be larger than any specific job's runtime in the workload.
- **MaxMemory**: Integer, in kilobytes. Again, this is the maximum the system allowed.
- **AllowOveruse**: Boolean. 'Yes' if a job may use more than it requested for any resource, 'No' if it can't.
- **MaxQueues**: Integer, number of queues used.
- **Queues**: A verbal description of the system's queues. Should explain the queue number field (if it has known values). As a minimum it should be explained how to tell between a batch and interactive job.
- **Queue**: A description of a single queue in the following format: queue-number queue-name (optional-details). This should be repeated for all the queues.
- **MaxPartitions**: Integer, number of partitions used.
- **Partitions**: A verbal description of the system's partitions, to explain the partition number field. For example, partitions can be distinct parallel machines in a cluster, or sets of nodes with different attributes (memory configuration, number of CPUs, special attached devices), especially if this is known to the scheduler.
- **Partition**: Description of a single partition.
- **Note**: There may be several notes, describing special features of the log. For example, The runtime is until the last node was freed; jobs may have freed some of their nodes earlier

In [32]:
import os


class Job:
    FIELDS_NAME = [
        "job",
        "submit",
        "wait",
        "runtime",
        "proc used",
        "cpu used",
        "mem used",
        "proc req",
        "cpu req",
        "mem req",
        "status",
        "uid",
        "gid",
        "executable",
        "queue",
        "partition",
        "prev jobs",
        "think time"
    ]
    
    FIELDS_SIZE = [
        10, 10, 6, 6, 3, 6, 10, 3, 10, 2, 2, 2, 2, 2, 2, 2, 2, 2
    ]
    
    def __init__(self, description):
        params = description.split()

        if len(params) != len(self.FIELDS_NAME):
            raise Exception("Bad len of job description")

        self.fields = {}

        for i in range(len(params)):
            self.fields[self.FIELDS_NAME[i]] = int(params[i])
    
    def pretty_print(self):
        print("===== Job Information =====")
        
        job_info = ""
        for field_name in self.FIELDS_NAME:
            job_info += "\t%s: %d\n" % (field_name, self.fields[field_name])
            
            
        print(job_info)
        
    @staticmethod
    def print_fields_name():
        max_len = max(map(len, Job.FIELDS_NAME))
        
        for i in range(max_len):
            line = ""
            for j in range(len(Job.FIELDS_NAME)):
                str_fmt = "%%%ds|" % Job.FIELDS_SIZE[j]
                ch = ""
                if (i+1) <= len(Job.FIELDS_NAME[j]):
                    ch = Job.FIELDS_NAME[j][i]
                line += str_fmt % ch

            print(line)

    def __str__(self):
        job_info = ""
        for i in range(len(self.FIELDS_NAME)):
            str_fmt = "%%%dd" % self.FIELDS_SIZE[i]
            job_info += ' ' + str_fmt % (self.fields[self.FIELDS_NAME[i]])

        return job_info


class LogFile:
    def __init__(self, path):
        self.path = path
        
        self.total_jobs = 0

        # count bad things
        self.cnt_fmt  = 0
        self.cnt_t0   = 0
        self.cnt_p0   = 0
        self.cnt_stat = 0
        self.cnt_bad  = 0

        # some useful globals
        self.start = 0
        self.timezone_str = ""
        self.max_jobs  = 0
        self.procs = 0
        self.nodes = 0
        
        self.jobs = []

    def read(self):
        file = open(self.path, "r")
        
        line = file.readline()
        while line != "":
            if line[0] == ';':
                if "UnixStartTime:" in line:
                    self.start = int(line.split(':')[1])
                elif "TimeZoneString:" in line:
                    self.timezone_str = line.split(':')[1]
                elif "MaxJobs:" in line:
                    self.max_jobs = int(line.split(':')[1])
                elif "MaxProcs:" in line:
                    self.procs = int(line.split(':')[1])
                elif "MaxNodes:" in line:
                    self.nodes = int(line.split(':')[1])

                line = file.readline()
                continue
                
            # Parse job line
            self.total_jobs += 1
            if self.total_jobs % 500000 == 0:
                print("Read '%s' jobs" % self.total_jobs)

            try:
                job = Job(line)
            except Exception as e:
                self.cnt_fmt += 1
                print("[ERROR] Cannot create job from '%s': '%s'" % (line, e))
                line = file.readline()
                continue

            if job.fields["runtime"] == 0:
                # something potentially fishy, as job took 0 time
                # but this can also be a resolution problem
                self.cnt_t0 += 1
                line = file.readline()
                continue

            if job.fields["proc used"] == 0:
                # something really fishy: job did not use any processors.
                # could mean job was cancelled before running
                self.cnt_p0 += 1
                line = file.readline()
                continue

            if job.fields["submit"] == -1 or job.fields["runtime"] == -1 or job.fields["proc used"] == -1:
                # something very fishy: job arrival, runtime or processors undefined
                self.cnt_bad += 1
                line = file.readline()
                continue
                
            if job.fields["status"] != 1:
                # another fishy:
                # job failed (status 0)
                # job was cancelled (status 5)
                # or job is only part of a whole job (status 2, 3, 4)
                self.cnt_stat += 1
                line = file.readline()
                continue

            self.jobs.append(job)
            line = file.readline()
            continue
        
        file.close()
   
    def print_jobs(self, pretty = True, amount = 10):
        print("LOG FILE '%s'" % self.path)
        Job.print_fields_name()
        
        if pretty == True:
            for i in range(0, min(len(self.jobs), amount)):
                self.jobs[i].pretty_print()
        else:
            for i in range(0, min(len(self.jobs), amount)):
                print(self.jobs[i])
                
        print('\n')
            
    def print_errors(self):
        print("Errors while read log file:")
        print("\t%d lines had a bad format" % self.cnt_fmt)
        print("\t%d jobs had 0 time" % self.cnt_t0)
        print("\t%d jobs had 0 processors" % self.cnt_p0)
        print("\t%d jobs had non-1 status" % self.cnt_stat)
        print("\t%d jobs had bad data \
              (undefined arrival, runtime or processors)" % \
              self.cnt_bad)

    @property
    def correct_jobs(self):
        return len(self.jobs)

    
# ============================== READ LOGS ==============================
LOG_DIRECTORY="../../parallel_workload_archive/log"
LOG_FILENAMES = [
    "CIEMAT-Euler-2008-1.swf"
]

LOG_FILES = []

for filename in LOG_FILENAMES:
    path = os.path.join(LOG_DIRECTORY, filename)
    
    print("Start read '%s'" % path)
    
    if not os.path.isfile(path):
        print("[ERROR] File '%s' is not exists" % path)
        continue

    log_file = LogFile(path)
    
    try:
        log_file.read()
    except Exception as e:
        print("[ERROR] Error while read log file '%s': %s" % (filename, e))
        continue

    log_file.print_errors()
    print("'%30s' was read successfully: TOTAL: %10d CORRECT: %10d" % \
          (filename, log_file.total_jobs, log_file.correct_jobs))
    LOG_FILES.append(log_file)

print('\n')
    
for log_file in LOG_FILES:
    log_file.print_jobs(pretty=True, amount=20)


Start read '../../parallel_workload_archive/log/CIEMAT-Euler-2008-1.swf'
Read '500000' jobs
Read '1000000' jobs
Read '1500000' jobs
Read '2000000' jobs
Read '2500000' jobs
Read '3000000' jobs
Read '3500000' jobs
Read '4000000' jobs
Read '4500000' jobs
Read '5000000' jobs
Read '5500000' jobs
Read '6000000' jobs
Read '6500000' jobs
Read '7000000' jobs
Read '7500000' jobs
Read '8000000' jobs
Read '8500000' jobs
Read '9000000' jobs
Errors while read log file:
	0 lines had a bad format
	1232994 jobs had 0 time
	0 jobs had 0 processors
	1681752 jobs had non-1 status
	4632 jobs had bad data               (undefined arrival, runtime or processors)
'       CIEMAT-Euler-2008-1.swf' was read successfully: TOTAL:    9263012 CORRECT:    6343634


LOG FILE '../../parallel_workload_archive/log/CIEMAT-Euler-2008-1.swf'
         j|         s|     w|     r|  p|     c|         m|  p|         c| m| s| u| g| e| q| p| p| t|
         o|         u|     a|     u|  r|     p|         e|  r|         p| e| t| i| i

In [33]:
# Print detail information about specific logs
# TODO
print("Number of jobs -> %d" % len(LOG_FILES[0].jobs))

Number of jobs -> 6343634


In [34]:
for log_file in LOG_FILES:
    log_file.print_jobs(pretty=False, amount=20)

LOG FILE '../../parallel_workload_archive/log/CIEMAT-Euler-2008-1.swf'
         j|         s|     w|     r|  p|     c|         m|  p|         c| m| s| u| g| e| q| p| p| t|
         o|         u|     a|     u|  r|     p|         e|  r|         p| e| t| i| i| x| u| a| r| h|
         b|         b|     i|     n|  o|     u|         m|  o|         u| m| a| d| d| e| e| r| e| i|
          |         m|     t|     t|  c|      |          |  c|          |  | t|  |  | c| u| t| v| n|
          |         i|      |     i|   |     u|         u|   |         r| r| u|  |  | u| e| i|  | k|
          |         t|      |     m|  u|     s|         s|  r|         e| e| s|  |  | t|  | t| j|  |
          |          |      |     e|  s|     e|         e|  e|         q| q|  |  |  | a|  | i| o| t|
          |          |      |      |  e|     d|         d|  q|          |  |  |  |  | b|  | o| b| i|
          |          |      |      |  d|      |          |   |          |  |  |  |  | l|  | n| s| m|
          |         

In [37]:
for log_file in LOG_FILES:
    print("[%s] TOTAL JOBS -> %10d CORRECT -> %10d UNIQUE -> %10d" %
          (log_file.path, log_file.total_jobs, log_file.correct_jobs,
           len(set(job.fields["job"] for job in log_file.jobs))))

[../../parallel_workload_archive/log/CIEMAT-Euler-2008-1.swf] TOTAL JOBS ->    9263012 CORRECT ->    6343634 UNIQUE ->    6343634
